In [1]:
import pandas as pd
import numpy as np
import random
import fasttext
import certifi
import ssl
import urllib.request
import os
import gzip
import spacy
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
import subprocess
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV

# seed = 2024
# random.seed(seed)
# np.random.seed(seed)

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # the expression of URL_pattern
    url_pattern = r'https?://\S+|www\.\S+'
    # use re.sub()to sustitude URL with blank str
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

In [5]:
#clean the data
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 

#convert categorical to numerical 
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})

# Load dataset
y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7650/280474677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7650/280474677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_7650/280474677.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [5]:
pip install PyICU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyICU: filename=PyICU-2.13.1-cp311-cp311-macosx_10_9_universal2.whl size=708994 sha256=f5f1e81865f2eabd714a5ce100c9e46739bb0a4f3b29125bece02c7c3f57910f
  Stored in directory: /Users/rachael/Library/Caches/pip/wheels/f7/40/2e/f5fa8292cf2624cd36da4b648670f6bd72132d75a953561e02
Successfully built PyICU

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 342.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp311-cp311-macosx_13_0_universal2.whl size=670562 sha256=ffdde8db49c7e3a83f21f77a4d70121ca724ffecebb52e74ec7dcbbb39fe6889
  Stored in directory: /Users/rachael/Library/Caches/pip/wheels/9c/42/d0/359c8cbd57e79a065dc3acf282270fccc57fdba07b307a299b
Successfully built fasttext
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import fasttext

# Path to your FastText pre-trained word embedding file
fasttext_model_path = '/Users/rachael/Downloads/cc.nl.300.bin'

# Load the FastText model
ft = fasttext.load_model(fasttext_model_path)

# Example usage: Get the vector for a word
# word = "voorbeeld"
# word_vector = ft.get_word_vector(word)

# print(f"Vector for the word '{word}':\n{word_vector}")

In [30]:
import sys
print(sys.executable)
!pip show PyICU

/usr/local/bin/python3
Name: PyICU
Version: 2.13.1
Summary: Python extension wrapping the ICU C++ API
Home-page: https://gitlab.pyicu.org/main/pyicu
Author: Andi Vajda
Author-email: vajda@pyicu.org
License: MIT
Location: /Users/rachael/opt/anaconda3/lib/python3.9/site-packages
Requires: 
Required-by: 


In [26]:
!brew install icu4c

zsh:1: command not found: brew


In [6]:

nlp = spacy.load("nl_core_news_sm")
#tokenize
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

def sentence_to_avg_vec(tokens, model):
    vector_size = model.get_dimension()
    vec = np.zeros(vector_size)
    count = 0
    for token in tokens:
        vec += model.get_word_vector(token)
        count += 1
    if count != 0:
        vec /= count
    return vec

In [12]:
pip install certifi


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
param_dist= {
    'n_estimators': range(10, 101),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 51),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

test_scores = []
best_params_list = []

# Stratified K-fold for maintaining label distribution,shuffle=True确保每次迭代的数据划分不同
# Initialize parameters
best_score = 0
best_params = {}
f1_scores = []

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]
    
    # convert y_train and y_test as DataFrame
    y_train_df = pd.DataFrame(trainy).reset_index(drop=True)
    y_test_df = pd.DataFrame(testy).reset_index(drop=True)

    # concat X_train and y_train, X_test and y_test
    train_df = pd.concat([trainX.reset_index(drop=True), y_train_df], axis=1)
    test_df = pd.concat([testX.reset_index(drop=True), y_test_df], axis=1)

    #prepare the data for data augmentation
    train_df['content'] = train_df['content'].apply(lambda x: ' '.join(x.split()))
    sentiment_0 = train_df[train_df['sentiment'] == 0]
    sentiment_1 = train_df[train_df['sentiment'] == 1]
    sentiment_2 = train_df[train_df['sentiment'] == 2]

    # the path to preserve prepared data for EDA data augmentation for class0 
    output_file_path0 = '/Users/rachael/Desktop/data/sentiment_0.txt'
    #convert DataFrame data into a simple tab-separated text file format，required by the data augmentation technique
    with open(output_file_path0, 'w', encoding='utf-8') as f:
        for _, row in sentiment_0.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path0,
    '--output', '/Users/rachael/Desktop/data/augmented_train0.txt',  # Output as txt if required
    '--num_aug', '9',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])

    #for class1
    output_file_path1 = '/Users/rachael/Desktop/data/sentiment_1.txt'
    with open(output_file_path1, 'w', encoding='utf-8') as f:
        for _, row in sentiment_1.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n") 
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path1,
    '--output', '/Users/rachael/Desktop/data/augmented_train1.txt',  # Output as txt if required
    '--num_aug', '1',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    
    #for class2
    output_file_path2 = '/Users/rachael/Desktop/data/sentiment_2.txt'
    with open(output_file_path2, 'w', encoding='utf-8') as f:
        for _, row in sentiment_2.iterrows():
            f.write(f"{row['sentiment']}\t{row['content']}\n")  
    subprocess.run([
    'python', '/Users/rachael/Desktop/data/augment.py',
    '--input', output_file_path2,
    '--output', '/Users/rachael/Desktop/data/augmented_train2.txt',  # Output as txt if required
    '--num_aug', '4',
    '--alpha_sr', '0.1',
    '--alpha_rd', '0.1',
    '--alpha_ri', '0.1',
    '--alpha_rs', '0.1'
])
    
    # Load augmented data
    file4 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train0.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file5 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train1.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    file6 = pd.read_csv('/Users/rachael/Desktop/data/augmented_train2.txt', delimiter='\t', header=None, names=['sentiment', 'content'])
    augmented_train = pd.concat([file4,file5,file6])

    #feature extraction for training data
    #augmented_train['content'] = augmented_train['content'].astype(str)
    augmented_train['content'] = augmented_train['content'].astype(str)
    train_sentences = [item for item in list(augmented_train['content'])]
    train_labels = [int(item) for item in list(augmented_train['sentiment'])]
    X_train_tokenized = [tokenize(sentence) for sentence in train_sentences]
    train_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_train_tokenized])

    ##prepare test data for models
    test_df['content'] = test_df['content'].astype(str)
    test_sentences = [item for item in list(test_df['content'])]
    test_labels = [int(item) for item in list(test_df['sentiment'])]
    X_test_tokenized = [tokenize(sentence) for sentence in test_sentences]
    test_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_test_tokenized])
    #feature extraction for test data

    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    rf = RandomForestClassifier(random_state=42)
    randomized_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42
    )

    # Fit Randomized Search
    randomized_search.fit(train_vectors,train_labels)

    # Get the best model from Randomized Search
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # Evaluate on the test set
    test_predictions = best_model.predict(test_vectors)
    test_f1_score = f1_score(test_labels, test_predictions, average='macro')

    f1_scores.append(test_f1_score)
    best_params_list.append(best_params)

    print(f"Fold F1-score: {test_f1_score}")
    print(f"Best parameters: {best_params}")

# Summarize the results
mean_f1_score = np.mean(f1_scores)
std_f1_score = np.std(f1_scores)

print(f"Mean F1-score: {mean_f1_score}")
print(f"Standard deviation of F1-scores: {std_f1_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_2.txt to /Users/rachael/Desktop/data/augmented_train2.txt with num_aug=4
Fold F1-score: 0.34488888888888886
Best parameters: {'n_estimators': 90, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 11, 'criterion': 'gini', 'bootstrap': False}
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_0.txt to /Users/rachael/Desktop/data/augmented_train0.txt with num_aug=9
generated augmented sentences with eda for /Users/rachael/Desktop/data/sentiment_1.txt to /Users/rachael/Desktop/data/augmented_train1.txt with num_aug=1
generated augmented sentence

In [9]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 11, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 90}


In [7]:
most_common_params = {'n_estimators': 46, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 28, 'criterion': 'gini', 'bootstrap': False}


In [8]:
X_sentences = [item for item in list(X)]
X_tokenized = X.apply(tokenize)
X_embeddings = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_tokenized])

final_model = RandomForestClassifier(random_state=42,**most_common_params,class_weight='balanced')
final_model.fit(X_embeddings, y)

RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=28,
                       min_samples_split=8, n_estimators=46, random_state=42)

In [9]:
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
#the data verified by the third person will be used as the train set finally
test = test[['content','sentiment']]
test['content'] = test['content'].apply(remove_urls) 
test['content'] = test['content'].apply(clean_text) 
test['sentiment'] = test['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
test_tokenized = test['content'].apply(tokenize)
test_embedding = np.array([sentence_to_avg_vec(sentence,ft) for sentence in test_tokenized])
test_predictions = final_model.predict(test_embedding)
test_score = accuracy_score(test['sentiment'], test_predictions)
conf_matrix = confusion_matrix(test['sentiment'], test_predictions)
class_report = classification_report(test['sentiment'], test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.63      0.97      0.76       245
           2       0.80      0.16      0.27       124

    accuracy                           0.63       407
   macro avg       0.48      0.38      0.34       407
weighted avg       0.62      0.63      0.54       407

